<a href="https://colab.research.google.com/github/Aditya757/MyRepository/blob/main/project_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/drive/MyDrive/peta.zip

Archive:  /content/drive/MyDrive/peta.zip
   creating: peta/
   creating: peta/test/
   creating: peta/Train/
   creating: peta/Train/Jacket/
  inflating: peta/Train/Jacket/0008.png  
  inflating: peta/Train/Jacket/0011_006.jpg  
  inflating: peta/Train/Jacket/0011_007.jpg  
  inflating: peta/Train/Jacket/0011_009.jpg  
  inflating: peta/Train/Jacket/0016.png  
  inflating: peta/Train/Jacket/0016_005.jpg  
  inflating: peta/Train/Jacket/0016_006.jpg  
  inflating: peta/Train/Jacket/0017_004.jpg  
  inflating: peta/Train/Jacket/0017_010.jpg  
  inflating: peta/Train/Jacket/0021_005.jpg  
  inflating: peta/Train/Jacket/0021_011.jpg  
  inflating: peta/Train/Jacket/0022_005.jpg  
  inflating: peta/Train/Jacket/0022_006.jpg  
  inflating: peta/Train/Jacket/0024.png  
  inflating: peta/Train/Jacket/0025_001.jpg  
  inflating: peta/Train/Jacket/0025_002.jpg  
  inflating: peta/Train/Jacket/0025_003.jpg  
  inflating: peta/Train/Jacket/0025_006.jpg  
  inflating: peta/Train/Jacket/0026_001.jp

In [2]:
pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 1.6 MB 66.5 MB/s 
     |████████████████████████████████| 5.8 MB 52.6 MB/s 
     |████████████████████████████████| 438 kB 74.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [3]:
import tensorflow as tf
tf.__version__

'2.9.1'

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [6]:
# re-size all the images to this
IMAGE_SIZE = (224, 224, 3)

train_path = '/content/peta/Train'
valid_path = '/content/peta/Validation'

In [7]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = InceptionV3(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 0s 0us/step


In [8]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [9]:
  # useful for getting number of output classes
folders = glob('/content/peta/Train/*')

In [10]:
folders

['/content/peta/Train/Long Hair',
 '/content/peta/Train/T-Shirt',
 '/content/peta/Train/Short Hair',
 '/content/peta/Train/Shorts',
 '/content/peta/Train/Shirt',
 '/content/peta/Train/Pant',
 '/content/peta/Train/Jacket']

In [11]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [13]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [14]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/peta/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
type(training_set)

Found 901 images belonging to 7 classes.


keras.preprocessing.image.DirectoryIterator

In [34]:
test_set = test_datagen.flow_from_directory('/content/peta/Validation',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')
type(test_set)

Found 491 images belonging to 7 classes.


keras.preprocessing.image.DirectoryIterator

In [32]:
# fit the model
# Run the cell. It will take some time to execute
validation_data=test_set
epochs=10
steps_per_epoch=len(training_set)
validation_steps=len(test_set)
r = model.fit(
  training_set,
  validation_data,
  epochs,
  steps_per_epoch,
  validation_steps
)

ValueError: ignored